In [3]:
%pylab inline

import pandas as pd
import sktensor

Populating the interactive namespace from numpy and matplotlib


In [4]:
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)

In [7]:
import ncp

#import importlib

#moduleName = input('ncp.py')
#importlib.import_module(ncp.py)

ModuleNotFoundError: No module named 'ncp.py'; 'ncp' is not a package

In [ ]:
classes = pd.read_csv("data/classes.csv", index_col=0, squeeze=True)
print(classes.size)
print(classes.unique() )

The data collected through the SocioPatterns platorm is represented as triples $<t, i, j>$ where $i,j$ is a pair of nodes that were in proximity during the interval between time $t$ and $t+20$.

It represents 2 days of activity in a school. We selected only the hours when the kids were at school, for a total of 18 hours (two intervals of 9 hours per day concatenated as a single interval, thus removing the night).

In [ ]:
data = pd.read_csv("data/school.csv.gz", compression="gzip")
print(data.head())
print(data.tail())
print(data.shape)

In [ ]:
print( data.t.max() / 3600., "hours")

From the timeline that shows the sum of number of contacts for each 20 seconds interval, we see that there is a small interval with no activity between the two 9 hours interval, in order to show where one day ends and starts the other.

In [ ]:
data.groupby(data.t).t.count().plot(linestyle='None', marker='.', figsize=(15,4));

For the purpose of this experiment, we divide the timeline in 150 different intervals.

In [ ]:
NR_NODES = classes.index.unique().shape[0]
NR_INTERVALS = 150

print(NR_NODES)

In [ ]:
data_binned = data.groupby((data.t / ((data.t.max()+1) / float(NR_INTERVALS))).astype(int))
print(data_binned.head(20))

In [ ]:
def create_adj(g):
    return g.groupby(['i','j']).size()

school_adj_series = data_binned.apply(create_adj)
school_adj_series.head(20)

We transform the $<t, i, j>$ triples in a 3-way binary tensor.

In [ ]:
triples = array([(i, j, t) for t, i, j in school_adj_series.keys()] + 
                [(j, i, t) for t, i, j in school_adj_series.keys()])
print(triples[1:10], triples[-10:-1])

X = sktensor.sptensor(tuple(triples.T), ones(len(triples)),
                      shape=(NR_NODES, NR_NODES, NR_INTERVALS))
print(X.shape)

The tensor is factorized using the alternating non-negative least squares algorithm with the block principal pivoting method, proposed in [[2]](#2).

In [ ]:
np.random.seed(4)

X_approx_ks = ncp.nonnegative_tensor_factorization(X, 14, method='anls_bpp')